<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [99]:
df1 = pd.read_csv('zip_train.csv', header=None)
print(df1)
type

      0    1    2    3      4      5    ...    251    252    253    254    255  256
0       6 -1.0 -1.0 -1.0 -1.000 -1.000  ... -0.474 -0.991 -1.000 -1.000 -1.000 -1.0
1       5 -1.0 -1.0 -1.0 -0.813 -0.671  ...  0.762  0.126 -0.095 -0.671 -0.828 -1.0
2       4 -1.0 -1.0 -1.0 -1.000 -1.000  ...  1.000 -0.179 -1.000 -1.000 -1.000 -1.0
3       7 -1.0 -1.0 -1.0 -1.000 -1.000  ... -1.000 -1.000 -1.000 -1.000 -1.000 -1.0
4       3 -1.0 -1.0 -1.0 -1.000 -1.000  ...  0.791  0.439 -0.199 -0.883 -1.000 -1.0
...   ...  ...  ...  ...    ...    ...  ...    ...    ...    ...    ...    ...  ...
7286    3 -1.0 -1.0 -1.0 -0.988 -0.527  ... -1.000 -1.000 -1.000 -1.000 -1.000 -1.0
7287    3 -1.0 -1.0 -1.0 -0.990  0.708  ... -1.000 -1.000 -1.000 -1.000 -1.000 -1.0
7288    3 -1.0 -1.0 -1.0 -0.783 -0.984  ... -0.342 -0.933 -1.000 -1.000 -1.000 -1.0
7289    0 -1.0 -1.0 -1.0 -1.000 -1.000  ... -0.231 -0.687 -1.000 -1.000 -1.000 -1.0
7290    1 -1.0 -1.0 -1.0 -1.000 -1.000  ... -1.000 -1.000 -1.000 -1.000 -1.0

type

In [103]:
df2 = df1.iloc[:,0]
print(df2)

0       6
1       5
2       4
3       7
4       3
       ..
7286    3
7287    3
7288    3
7289    0
7290    1
Name: 0, Length: 7291, dtype: int64


In [105]:
df1 = df1.drop([0],axis=1)
print(df1)

      1    2    3      4      5    ...    252    253    254    255  256
0    -1.0 -1.0 -1.0 -1.000 -1.000  ... -0.991 -1.000 -1.000 -1.000 -1.0
1    -1.0 -1.0 -1.0 -0.813 -0.671  ...  0.126 -0.095 -0.671 -0.828 -1.0
2    -1.0 -1.0 -1.0 -1.000 -1.000  ... -0.179 -1.000 -1.000 -1.000 -1.0
3    -1.0 -1.0 -1.0 -1.000 -1.000  ... -1.000 -1.000 -1.000 -1.000 -1.0
4    -1.0 -1.0 -1.0 -1.000 -1.000  ...  0.439 -0.199 -0.883 -1.000 -1.0
...   ...  ...  ...    ...    ...  ...    ...    ...    ...    ...  ...
7286 -1.0 -1.0 -1.0 -0.988 -0.527  ... -1.000 -1.000 -1.000 -1.000 -1.0
7287 -1.0 -1.0 -1.0 -0.990  0.708  ... -1.000 -1.000 -1.000 -1.000 -1.0
7288 -1.0 -1.0 -1.0 -0.783 -0.984  ... -0.933 -1.000 -1.000 -1.000 -1.0
7289 -1.0 -1.0 -1.0 -1.000 -1.000  ... -0.687 -1.000 -1.000 -1.000 -1.0
7290 -1.0 -1.0 -1.0 -1.000 -1.000  ... -1.000 -1.000 -1.000 -1.000 -1.0

[7291 rows x 256 columns]


In [107]:
Xu = df1.to_numpy()
Yu = df2.to_numpy()
print(Xu)
print(Yu.T)

[[-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -0.671 -0.828 -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 ...
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]
 [-1.    -1.    -1.    ... -1.    -1.    -1.   ]]
[6 5 4 ... 3 0 1]


In [108]:
model = LinearRegression().fit(Xu, Yu)
#https://realpython.com/linear-regression-in-python/

r_sq = model.score(Xu, Yu)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

coefficient of determination: 0.6553555286479579
intercept: 0.6094210785273124
slope: [-2.10204907e+00  9.11976830e-02 -1.63988603e-01 -1.23320599e-02
 -2.10291265e-01  7.71033425e-02 -1.71900609e-01 -5.27095869e-01
  1.28543009e-01  5.47142038e-01  3.90356555e-01  1.50661651e-02
  1.95101882e-01  4.20025760e-01  2.54763569e-02  8.47176256e-01
 -4.20234394e-01 -5.87115069e-01 -2.42292640e-01 -2.65172432e-01
 -1.71545484e-01 -7.98128629e-02  1.50056958e-02 -6.05281810e-03
 -3.28221479e-02  2.43497929e-01  1.70156810e-01  1.73360631e-01
  2.72244427e-01 -6.53270061e-02  5.34634326e-01 -4.51186945e-02
 -2.22873743e-01  2.40298241e-01 -2.11435732e-01 -8.18247007e-02
 -2.20919169e-01  2.78927464e-02 -6.65896910e-02 -2.94842798e-01
 -4.89675664e-02 -1.01729134e-01  2.10383509e-01  2.19651651e-02
  1.99255869e-01  3.34706114e-01  4.88233042e-01  3.24227357e-01
 -1.86363171e-01 -4.47216268e-01 -1.23522317e-01 -1.77574690e-02
 -1.79096816e-02  6.65981186e-02 -1.24428963e-01 -1.20515241e-01
 -1.

In [109]:
y_pred = model.predict(Xu)
print('predicted response:', y_pred, sep='\n')

predicted response:
[2.17680163 3.92892234 2.73468605 ... 3.10574952 0.10858921 4.2994831 ]
